In [2]:
import math
from projectq import MainEngine
from projectq.ops import H, Z, X, Measure, All, XOR
from projectq.meta import Loop, Compute, Uncompute, Control

In [6]:
eng=MainEngine()

In [58]:
def D(eng,x):
    # the example given by the projectQ documentation is really wired
    # how does this work?
    '''
    implement grover diffusion gate
    verification is given by gate decomposition
    '''
    with Compute(eng):
        All(H) | x
        All(X) | x
    with Control(eng, x[0:-1]):
        Z | x[-1]
    Uncompute(eng)

In [80]:
def O(eng,x,b):
    # how does this work???
    with Compute(eng):
        X| x[3] # determine the target bit 
    with Control(eng,x):
        X | b
    Uncompute(eng)

In [81]:
def grover_search(eng,n,O_func):
    init_X=eng.allocate_qureg(n)
    All(H)|init_X # superposition
    ancillary_B=eng.allocate_qubit()
    X|ancillary_B # input |1> state
    H|ancillary_B # get |-> state
    
    iter_n=int(math.pi/4.*math.sqrt(1 << n))
    # apply Oracle gate and Grover Diffusion gate 
    with Loop(eng,iter_n):
        O(eng,init_X,ancillary_B)
        D(eng,init_X)
    
    # measure the result, large probability to reach the target bit
    All(Measure) | init_X
    Measure | ancillary_B
    eng.flush()
    
    return [int(x) for x in init_X]

In [83]:
if __name__ == "__main__":
    eng = MainEngine()
    print(grover_search(eng, 8, O))

[1, 1, 1, 0, 1, 1, 1, 1]
